In [1]:
%reload_ext autoreload
%autoreload 2
from MTPHandler.core import Plate
from MTPHandler.readers import read_magellan

from EnzymePynetics.core import Estimator

## Standard curve for ABTS radical

In [2]:
standard_plate = Plate.from_reader(
    reader=read_magellan,
    path="laccase_kinetics/abtscorrelation_rawdata.xlsx",
    ph=5.3,
    wavelength=420,
)

standard_plate.visualize(wavelengths=420, zoom=False)

In [3]:
radical = standard_plate.add_reactant(
    id="abts_radical", name="ABTS radical", constant=False
)

buffer = standard_plate.add_reactant(id="buffer", name="Buffer", constant=True)

In [4]:
standard_plate.assign_species(species=buffer, conc_unit="mM", init_conc=50, to="all")


standard_plate.assign_species_from_spreadsheet(
    species=radical,
    conc_unit="mmol / l",
    path="laccase_kinetics/abtscorrelation_rawdata.xlsx",
    sheet_name="Well assignment",
)

Assigned Buffer to all wells.


In [5]:
standard_plate.blank_species(buffer, 420)

Mean absorption of Buffer at 50.0 mM: -0.046035714285714284
Blanked 62 wells containing Buffer.


In [6]:
standard = standard_plate.calibrate(
    radical, 420, cutoff=2.9, signal_type="Absorbance")

In [7]:
standard.fit_models()

,AIC,R squared,RMSD
Model Name,,,
cubic,-243,0.9990,0.0316
quadratic,-230,0.9985,0.0387
linear,-182,0.9941,0.0777


In [8]:
standard.visualize()

In [9]:
# Save the best model for concentration calculation
standard = standard.save_model("cubic")

## Process kinetic data

In [10]:
plate_woTween = Plate.from_reader(
    reader=read_magellan,
    path="laccase_kinetics/Laccase_kinetic_woTween_Rawdata.xlsx",
    ph=5.3,
    wavelength=420,
)

plate_Tween = Plate.from_reader(
    reader=read_magellan,
    path="laccase_kinetics/Laccase_kinetic_wthTween_Rawdata.xlsx",
    ph=5.3,
    wavelength=420,
)

plates = [plate_woTween, plate_Tween]

plate_Tween.visualize(wavelengths=420, zoom=False)

In [11]:
for plate in plates:
    # Define laccase
    laccase = plate.add_protein(
        id="p0",
        name="Trametes versicolor laccase",
        constant=True,
        uniprotid="O13448",
        sequence="MSRFHSLLAFVVASLTAVAHAGIGPVADLTITNAAVSPDGFSRQAVVVNGGTPGPLITGNMGDRFQLNVIDNLTNHTMLKSTSIHWHGFFQKGTNWADGPAFINQCPISSGHSFLYDFQVPDQAGTFWYHSHLSTQYCDGLRGPFVVYDPNDPAADLYDVDNDDTVITLVDWYHVAANVGPAFPLGADATLINGKGRSPSTTTADLSVISVTPGKRYRFRLVSLSCDPNYTFSIDGHNMTIIETDSINTAPLVVDSIQIFAAQRYSFVLEANQAVDNYWIRANPNFGNVGFTGGINSAILRYDGAAAVEPTTTQTTSTEPLNEVNLHPLVATAVPGSPVAGGVDLAINMAFNFNGTNFFINGASFTPPTVPVLLQIISGAQNAQDLLPSGSVYSLPSNADIEISFPATAAAPGAPHPFHLHGHAFAVVRSAGSTVYNYDNPIFRDVVSTGTPAAGDNVTIRFRTDNPGPWFLHCHIDFHLEAGFAVVFAEDIPDVASANPVPQAWSDLCPTYDALDPSDQ",
    )

    plate.assign_species(
        species=laccase, conc_unit="mmol / l", init_conc=0.000351, to="all"
    )

    # Define ABTS radical product
    radical = plate.add_reactant(**radical.to_dict())
    plate.assign_species(
        species=radical,
        conc_unit="mmol / l",
        init_conc=0,
        to="all",
        contributes_to_signal=False,
    )

    # Define ABTS substrate
    abts = plate.add_reactant(
        id="s0",
        name="ABTS",
        constant=False,
    )

plate_woTween.assign_species_from_spreadsheet(
    species=abts,
    conc_unit="mmol / l",
    path="laccase_kinetics/Laccase_kinetic_woTween_Rawdata.xlsx",
    sheet_name="Well assignment",
)

plate_Tween.assign_species_from_spreadsheet(
    species=abts,
    conc_unit="mmol / l",
    path="laccase_kinetics/Laccase_kinetic_wthTween_Rawdata.xlsx",
    sheet_name="Well assignment",
)

Assigned Trametes versicolor laccase to all wells.
Assigned ABTS radical to all wells.
Assigned Trametes versicolor laccase to all wells.
Assigned ABTS radical to all wells.


/Users/max/Documents/GitHub/software-driven-rdm/sdRDM/base/datamodel.py:322: UserWarning:

No 'URL' and 'Commit' specified. This model might not be re-usable.



In [12]:
for plate in plates:
    plate.blank_species(laccase, 420)

Mean absorption of Trametes versicolor laccase at 0.000351 mmol / l: -0.0405725
Blanked 64 wells containing Trametes versicolor laccase.
Mean absorption of Trametes versicolor laccase at 0.000351 mmol / l: -0.04699
Blanked 72 wells containing Trametes versicolor laccase.


In [13]:
enzml_woTween = plate_woTween.to_enzymeml(
    name="Laccase kinetics without Tween",
    detected_reactant=radical,
    wavelength=420,
    reactant_standard=standard,
    sort_measurements_by=abts,
    ignore_blank_status=True,
)

enzml_Tween = plate_Tween.to_enzymeml(
    name="Laccase kinetics with Tween",
    detected_reactant=radical,
    wavelength=420,
    sort_measurements_by=abts,
    reactant_standard=standard,
    ignore_blank_status=True,
)

Found 64 catalyzed wells
Found 72 catalyzed wells


## Parameter estimation

In [15]:
estimator_woTween, _ = Estimator.from_enzymeml(enzml_woTween, radical)
estimator_Tween, _ = Estimator.from_enzymeml(enzml_Tween, radical)

estimators = [estimator_woTween, estimator_Tween]

In [16]:
for estimator in estimators:
    estimator.add_reaction(
        id="r0",
        name="ABTS oxidation",
        educt=abts,
        product=radical,
        catalyst=laccase,
    )

    estimator.add_model(
        id="m0",
        name="Michaelis-Menten",
        equation="substrate = -(k_cat * catalyst * substrate) / (K_M + substrate)",
    )

### Visualize measurement data

In [17]:
estimator_woTween.visualize_data()

In [18]:
estimator_Tween.visualize_data()

Based on the visualization of the measurement data, measurements form wells E12, F12, G12, H12, E11, H11, G11, A11, B11, C11, D11, C4, E1, F1, G1, and H1 were excluded from kinetic analysis.

In [19]:
excluded_replicates = [
    "E12",
    "F12",
    "G12",
    "H12",
    "E11",
    "H11",
    "G11",
    "A11",
    "B11",
    "C11",
    "D11",
    "C4",
    "E1",
    "F1",
    "G1",
    "H1",
]

for replicate_id in excluded_replicates:
    estimator_Tween.remove_replicate(replicate_id)

In [20]:
estimator_Tween.visualize_data()

In [21]:
for estimator in estimators:
    min_time = 300
    estimator.fit_models(min_time=min_time)
    estimator.correlations()
    estimator.visualize(min_time=min_time)
    estimator.model_table()

Fitting Michaelis-Menten model: 100%|██████████| 1/1 [00:01<00:00,  1.89s/it]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / s,mmol / l,1 / s / mmol / l,mmol / l,mmol / l,1 / s
Model,,,,,,,
Michaelis-Menten,-7101,1.099 ± 0 %,0.400 ± 1 %,2.747 ± 1 %,,,


Fitting Michaelis-Menten model: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / s,mmol / l,1 / s / mmol / l,mmol / l,mmol / l,1 / s
Model,,,,,,,
Michaelis-Menten,-6688,0.867 ± 0 %,1.049 ± 2 %,0.827 ± 2 %,,,


In [22]:
reaction_system = estimator_woTween.get_reaction_system("Michaelis-Menten")
estimator_woTween.to_enzymeml(
    enzml_woTween,
    reaction_system=reaction_system,
    out_path=f"{enzml_woTween.name}.json",
)

EnzymeMLDocument(id='enzymemldocument0', name='Laccase kinetics without Tween', pubmedid=None, url=None, doi=None, created=datetime.datetime(2024, 2, 5, 12, 57, 1, 683708), modified=None, creators=[], vessels=[Vessel(id='v0', name='MTP 96 well', volume=200.0, unit='ul', constant=True, uri=None, creator_id=None)], proteins=[Protein(id='p0', name='Trametes versicolor laccase', vessel_id='v0', init_conc=None, constant=True, unit=None, uri=None, creator_id=None, sequence='MSRFHSLLAFVVASLTAVAHAGIGPVADLTITNAAVSPDGFSRQAVVVNGGTPGPLITGNMGDRFQLNVIDNLTNHTMLKSTSIHWHGFFQKGTNWADGPAFINQCPISSGHSFLYDFQVPDQAGTFWYHSHLSTQYCDGLRGPFVVYDPNDPAADLYDVDNDDTVITLVDWYHVAANVGPAFPLGADATLINGKGRSPSTTTADLSVISVTPGKRYRFRLVSLSCDPNYTFSIDGHNMTIIETDSINTAPLVVDSIQIFAAQRYSFVLEANQAVDNYWIRANPNFGNVGFTGGINSAILRYDGAAAVEPTTTQTTSTEPLNEVNLHPLVATAVPGSPVAGGVDLAINMAFNFNGTNFFINGASFTPPTVPVLLQIISGAQNAQDLLPSGSVYSLPSNADIEISFPATAAAPGAPHPFHLHGHAFAVVRSAGSTVYNYDNPIFRDVVSTGTPAAGDNVTIRFRTDNPGPWFLHCHIDFHLEAGFAVVFAEDIPDVASANPVPQAWSDLCPTYDALDPSDQ', ecnu

In [23]:
reaction_system = estimator_Tween.get_reaction_system("Michaelis-Menten")
estimator_Tween.to_enzymeml(
    enzml_Tween,
    reaction_system=reaction_system,
    out_path=f"{enzml_Tween.name}.json",
)

EnzymeMLDocument(id='enzymemldocument1', name='Laccase kinetics with Tween', pubmedid=None, url=None, doi=None, created=datetime.datetime(2024, 2, 5, 12, 57, 3, 660851), modified=None, creators=[], vessels=[Vessel(id='v0', name='MTP 96 well', volume=200.0, unit='ul', constant=True, uri=None, creator_id=None)], proteins=[Protein(id='p0', name='Trametes versicolor laccase', vessel_id='v0', init_conc=None, constant=True, unit=None, uri=None, creator_id=None, sequence='MSRFHSLLAFVVASLTAVAHAGIGPVADLTITNAAVSPDGFSRQAVVVNGGTPGPLITGNMGDRFQLNVIDNLTNHTMLKSTSIHWHGFFQKGTNWADGPAFINQCPISSGHSFLYDFQVPDQAGTFWYHSHLSTQYCDGLRGPFVVYDPNDPAADLYDVDNDDTVITLVDWYHVAANVGPAFPLGADATLINGKGRSPSTTTADLSVISVTPGKRYRFRLVSLSCDPNYTFSIDGHNMTIIETDSINTAPLVVDSIQIFAAQRYSFVLEANQAVDNYWIRANPNFGNVGFTGGINSAILRYDGAAAVEPTTTQTTSTEPLNEVNLHPLVATAVPGSPVAGGVDLAINMAFNFNGTNFFINGASFTPPTVPVLLQIISGAQNAQDLLPSGSVYSLPSNADIEISFPATAAAPGAPHPFHLHGHAFAVVRSAGSTVYNYDNPIFRDVVSTGTPAAGDNVTIRFRTDNPGPWFLHCHIDFHLEAGFAVVFAEDIPDVASANPVPQAWSDLCPTYDALDPSDQ', ecnumbe